## Video Action Recognition R(2+1)D
- Paper : (2017) https://arxiv.org/abs/1711.11248
    - A Closer Look at Spatiotemporal Convolutions for Action Recognition
    
- 3D conv를 공간정보와 시간정보로 나눠 conv하는 구조를 처음 제안
    - 3D convolutional filters (N*t*d*d)를 (N*1*d*d) 와 (M*t*1*1)로 분해
    - 기존 3D conv와 파라미터 동일 / Non-linearities 증가 / Optimization 도움
- 다양한 spatiotemporal conv 방법들을 비교 
    - R2D F-R2D, MCx, rMCx, R3D, R(2+1)D
- RGB, Flow 두 부분에 대해 R(2+1)D 모델에 각각 학습 및 Prediction Averaging
- Sports-1m, kenetics, ucf101, hmdb51에서 state-of-the-art를 달성

<hr>

## 1. Abstract & Introduction
- 기존은 2d conv나 3d conv 나 비슷한 결과를 보이며 3d-conv에 대한 연구가 더딤

- 시간적 정보는 action recognition의 정확한 행동인식에 필수적이지 않다, 이미 정적 frame들이 action class에 대해 정보를 가지고 있기 때문
    - temporal reasoning is not essential for accurate action recognition, 
    - because of the strong action class information already contained in the static frames of a sequence.

- 3D Conv Filters -> separate spatial and temporal components
    - 3D conv를 두개의 개별적인 2D 공간 conv와 1D 시간 conv로 분해 하는 것
    - 필터를 별도의 공간 및 시간 구성 요소로 분해시 정확도가 크게 향상
    - a decomposition of 3D convolution into **a 2D spatial convolution followed by 1D temporal convolution**, which we name (2+1)D convolution

<hr>

### 1-1 Advantages
- 장점
    - (1) 파라메터가 증가하지 않음(2d 사용), relu를 두번 사용할 수 있기때문에 비선형성을 증가 (nonlinear rectification)
    - (2) Facilitates the Optimization (2+1)D 블록이 최적화 하기 더 쉬움 
        - the (2+1)D blocks (with factorized spatial and temporal components) are easier to optimize

<hr>


## 2. Convolutional residual blocks for video

![r2+1d_model](img/r2+1d_model.PNG)

### 2-1. R2D
- 2D convolutions (2D CNN 방법, 3L x H x W, 1프레임당 3채널씩 할당)
- 프레임을 채널로 쌓아서 사용하는 방법

### 2-2. f-R2D
- 2D convolutions over frames, f-R2D는 2D CNN방법
- 전체의 프레임을 개별적으로 2D CNN 하는 방법

<hr>



### 2-3. MCx and rMCx: mixed 3D-2D convolutions
- 3D convolution이 초기 계층에서 특히 유용할 수 있는 반면, 후반 계층에서는 필수적이지 않다의 생각
- (MCx) 학습 초기에는 3D Conv를사용 + 후반에는 2D conv 사용
- (rMCx) 학습 초기에는 2D Conv를 사용 + 후반에는 3D conv 사용

### 2-4. R3D: 3D convolutions
- R3D는 가장 일반적인 3D conv 방법 (3 x L x H X W)
- 프레임을 시간축을 하나 추가하여 stack하고 3D conv를 하는것 입니다.

<hr>

### 2-5. R(2+1)D: (2+1)D convolutions
- (좌측) R(2+1)D와 R3D간의 비교 
    - R(2+1)D eases the optimization as depth is increased
    - 시공간 분해가 유효함을 증명

- (우측) **Full 3D conv를 2D의 공간 conv와 1D 시간 conv로 분해**
    - **3D convolutional filters($N*t*d*d$)를 $N*1*d*d$ 와 $M*t*1*1$로 분해** 
    - 기존의 3D conv와 동일 및 Non-linearities 증가 시키고 Optimization에 도움을 줌
    - 하이퍼 파라미터 $M_i$ 는 공간과 공간 사이에 신호가 투영되는 중간 부분 공간의 차원을 결정

![r2+1d_factorization_r3d](img/r2+1d_factorization_r3d.png)

<hr>

## 3. Experiments
- 오버피팅이 나타나지 않기 위하여 충분히 큰 데이터셋인 Sport 1M, Kinetics을 사용 학습
- 작은 데이터셋에서 잘 작동하나 궁금하기 때문에, UCF101,HMDB51에 transfer learning을 적용
- 실험의 good performance and simplicity을 위해 18-Layer와 34-Layer 사용

### 3.1 Model Architecture

![r2+1d_model_architecture](img/r2+1d_model_architecture.PNG)

- R3D 18-Layer, 34-Layer 먼저 생성
- 112 × 112 크기의 L RGB 프레임으로 구성된 클립을 입력으로 사용
- conv1에서 1×2×2의 Conv stride에 의해 시공간 다운 샘플링
- conv3 1, conv4 1, conv5 1에서 2×2×2의 conv stride 3개 시공간 다운 샘플링 (=stride 이용 함)

- R2D, MCx, rMCx, R(2+1)D
    - 3D 컨볼 루션을 각각 2D Conv, Mixed Conv, Reversed Mixed Conv, (2+1)D Conv로 대체
- 시공간 GAP, Softmax (FC Layer)

### 3.2 Optical Flow - Two Stream
- RGB 및 Optical flow Input 모두에 대해 R(2+1)D 아키텍처 훈련
    - We train our R(2+1)D architecture on both RGB and optical flow inputs
- fuse the prediction scores by averaging
- Farneback의 방법 사용 Optical flow을 계산

<hr>


## 4. Training
- Kinetics 이용 학습
- Video frames scaled 128×171 -> randomly cropping 112×112.
- We randomly sample L consecutive frames with temporal jittering
- models are trained on 8-frame clips (L = 8) and 16-frame clips (L = 16)
- Batch normalization is applied to all convolutional layers
- initial learning rate is set to 0.01 and divided by 10 every 10 epochs
- caffe2 , SGD, GPU Cluster

![r2+1d_result](img/r2+1d_result.PNG)
- R2D부터 R(2+1)D까지 모든 net을 clip단위와 video단위로 평가
- Kinetics valid 데이터셋 accuracy 결과
- R(2+1)D 은 R3D와 Params 차이는 비슷,  Accuracy는 가장 높음

<hr>


## 5. Results
![r2+1d_result1](img/r2+1d_result1.png)

![r2+1d_result2](img/r2+1d_result2.png)

![r2+1d_result3](img/r2+1d_result3.png)

- 대부분의 지표가 높은 성능을 보임
- 단 I3D Two-Stream보다 낮은 이유를 보이는 건 I3D는 TV-L1, R(2+1)D 는 Farnebacks's optical flow라서 그렇다고 함.

## 참고문헌
- https://blog.airlab.re.kr/2019/09/R(2+1)D